In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [4]:
df = pd.read_csv('lancer.csv')
weapons = pd.read_json('weapons.json')

In [2]:
# Variables:
# License Level L (+L/2 rounded up base attack bonus)
# A accuracy (+ 1d6 (3.5 average) * A bonus)
# E target evasion (Meet or beat to score a hit)
# D difficulty (- 1d6 (3.5 average) * D bonus)
# H Hits (Increments by 1 each successful hit, modifies D)

In [20]:
weapons = weapons.drop(columns=['data_type'])

In [214]:
weap = weapons[weapons.name == 'COMBAT DRILL'].to_dict()
weap['id']

{38: 'mw_combat_drill'}

In [223]:
weapons['id'].to_list()

['mw_anti_materiel_rifle',
 'mw_assault_rifle',
 'mw_charged_blade',
 'mw_cyclone_pulse_rifle',
 'mw_heavy_charged_blade',
 'mw_heavy_machine_gun',
 'mw_heavy_melee_weapon',
 'mw_howitzer',
 'mw_missile_rack',
 'mw_mortar',
 'mw_nexus_hunter_killer',
 'mw_nexus_light',
 'mw_pistol',
 'mw_segment_knife',
 'mw_rocket_propelled_grenade',
 'mw_shotgun',
 'mw_tactical_knife',
 'mw_tactical_melee_weapon',
 'mw_thermal_lance',
 'mw_thermal_pistol',
 'mw_thermal_rifle',
 'mw_chain_axe',
 'mw_bristlecrown_flechette_launcher',
 'mw_nanocarbon_sword',
 'mw_assault_cannon',
 'mw_concussion_missiles',
 'mw_leviathan_heavy_assault_cannon',
 'mw_cutter_mkii_plasma_torch',
 'mw_war_pike',
 'mw_power_knuckles',
 'mw_hand_cannon',
 'mw_bolt_thrower',
 'mw_kinetic_hammer',
 'mw_deck_sweeper_automatic_shotgun',
 'mw_daisy_cutter',
 'mw_catalytic_hammer',
 'mw_impact_lance',
 'mw_impaler_nailgun',
 'mw_combat_drill',
 'mw_magnetic_cannon',
 'mw_vulture_dmr',
 'mw_railgun',
 'mw_veil_rifle',
 'mw_burst_laun

In [176]:
def has_tag(weapon, tag):
    weap_id = list(weapon['id'].keys())[0]
    return any([tag['id'] == 'tg_overkill' for tag in weapon['tags'][weap_id]])

In [118]:
def attack_roll(license, evasion, acc=0, diff=0):
    if acc > diff:
        acc -= diff
    elif diff > acc:
        diff -= acc
    else:
        diff, acc = 0, 0
    result = np.random.randint(1, 21) + np.ceil((license/2)) + (acc*np.random.randint(1, 7)) - (diff*np.random.randint(1, 7))
    return result, result >= evasion

In [219]:
def attack(weapon, license, evasion, acc=0, diff=0):
    weap_id = list(weapon['id'].keys())[0]
    damages = [dam['val'] for dam in weapon['damage'][weap_id]]
    nums = [str(dice).split('d') for dice in damages]
    bons = [num[1].split('+') if len(num) > 1 else [0, num[0]] for num in nums]
    dice = [int(bon[0]) for bon in bons]
    mults = [int(num[0]) if len(num) > 1 else 0 for num in nums]
    bonus = sum([int(bon[1]) if len(bon) > 1 else 0 for bon in bons])
    
    accuracy = has_tag(weapon, 'tg_accurate')
    difficulty = has_tag(weapon, 'tg_inaccurate')
    if accuracy:
        acc += 1
    if difficulty:
        diff += 1
    
    to_hit = attack_roll(license, evasion, acc, diff)
    if to_hit[1]:
        i = 0
        rolls = []
        while i < len(dice):
            rolls += [np.random.randint(1, dice[i]+1) for _ in range(mults[i])]
            i += 1
        if has_tag(weapon, 'tg_overkill'):
            for roll in rolls:
                if roll == 1:
                    rolls.append(np.random.randint(1, 6+1))
                    if weapon['id'] == {38: 'mw_combat_drill'}:
                        rolls.append(np.random.randint(1, 6+1))
        damage = sum(rolls) + bonus
    else:
        damage = 0
        for tag in weapon['tags'][weap_id]:
            if tag['id'] == 'tg_reliable':
                damage = tag['val']
    return damage

In [217]:
def sample(weapon, license, evasion, acc=0, diff=0):
    return [attack(weapon, license, evasion, acc, diff) for _ in range(50)]

In [193]:
weap = weapons[weapons.id == 'mw_combat_drill'].to_dict()
attack(weap, 3, 10)

17

In [ ]:
weapon_list = []
damage = []
license_level = []
target_evasion = []
accuracy = []
difficulty = []


In [228]:
ll = 0
te = 5
acc = 0
diff = 0
i = 0
while i < len(weapons['id']):
    weapons_list = weapons['id'].to_list()
    weap = weapons[weapons.id == weapons_list[i]].to_dict()
    print(weap['id'])
    diff = 0
    while diff < 7:
        acc = 0
        while acc < 7:
            ll = 0
            while ll < 13:
                te = 5
                while te < 21:
                    barrage = sample(weap, ll, te, acc, diff)
                    for skirmish in barrage:
                        weapon_list.append(list(weap['id'].values())[0])
                        damage.append(skirmish)
                        license_level.append(ll)
                        target_evasion.append(te)
                        accuracy.append(acc)
                        difficulty.append(diff)
                    te += 1
                ll += 1
            acc += 1
        diff += 1
    i += 1

{84: 'mw_sherman_integrated'}


ValueError: invalid literal for int() with base 10: '6/Charge'

In [221]:
df = pd.DataFrame({'Weapon': weapon_list, 'Damage': damage, 'License_Level': license_level, 'Target_Evasion': target_evasion, 'Accuracy': accuracy, 'Difficulty': difficulty})

In [230]:
df

,Weapon,Damage,License_Level,Target_Evasion,Accuracy,Difficulty
0,mw_anti_materiel_rifle,0,0,5,0,0
1,mw_anti_materiel_rifle,9,0,5,0,0
2,mw_anti_materiel_rifle,0,0,5,0,0
3,mw_anti_materiel_rifle,4,0,5,0,0
4,mw_anti_materiel_rifle,7,0,5,0,0
...,...,...,...,...,...,...
32104795,mw_smartgun,0,12,20,6,6
32104796,mw_smartgun,4,12,20,6,6
32104797,mw_smartgun,0,12,20,6,6
32104798,mw_smartgun,4,12,20,6,6


In [231]:
df.to_csv("lancer.csv")

SystemError: <built-in method item of numpy.ndarray object at 0x11795c030> returned a result with an error set

In [234]:
plt.scatter(df['Weapon'], [])

TypeError: scatter() missing 1 required positional argument: 'y'